In [1]:
#import basic libraries final version
import pandas as pd
import numpy as np
# import numpy_indexed as npi

# #visualization imports
# import matplotlib as plt
# import seaborn as sns
# %matplotlib inline

#import sparse matrices
import scipy.sparse as sps

#import SkLearn
from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.linear_model import RandomizedLogisticRegression
# from sklearn.decomposition import MiniBatchDictionaryLearning

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# from collections import defaultdict


In [2]:
#import Instacart data
aisles = pd.read_csv('/Users/michaeljoyce/Desktop/Recommender_Engine_Instacart/data/aisles.csv')              
orders= pd.read_csv('/Users/michaeljoyce/Desktop/Recommender_Engine_Instacart/data/orders.csv') 
departments= pd.read_csv('/Users/michaeljoyce/Desktop/Recommender_Engine_Instacart/data/departments.csv')            
products= pd.read_csv('/Users/michaeljoyce/Desktop/Recommender_Engine_Instacart/data/products.csv') 
order_products_prior= pd.read_csv('/Users/michaeljoyce/Desktop/Recommender_Engine_Instacart/data/order_products__prior.csv')  
order_products_train= pd.read_csv('/Users/michaeljoyce/Desktop/Recommender_Engine_Instacart/data/order_products__train.csv') 

In [3]:
filenames=['aisles','orders','departments','products', 'order_products_prior', 'order_products_train']


In [4]:
# check the shape of each dataframe and check for any null values
for name in filenames:
    print(name, eval(name).shape, ":Null values?", eval(name).isnull().any())

aisles (134, 2) :Null values? aisle_id    False
aisle       False
dtype: bool
orders (3421083, 7) :Null values? order_id                  False
user_id                   False
eval_set                  False
order_number              False
order_dow                 False
order_hour_of_day         False
days_since_prior_order     True
dtype: bool
departments (21, 2) :Null values? department_id    False
department       False
dtype: bool
products (49688, 4) :Null values? product_id       False
product_name     False
aisle_id         False
department_id    False
dtype: bool
order_products_prior (32434489, 4) :Null values? order_id             False
product_id           False
add_to_cart_order    False
reordered            False
dtype: bool
order_products_train (1384617, 4) :Null values? order_id             False
product_id           False
add_to_cart_order    False
reordered            False
dtype: bool


In [5]:
#combine order_products_train, products, departments, and aisles

opt1=order_products_train.merge(products,how='left', on='product_id')
opt2=opt1.merge(departments,how='left', on='department_id')
opt3=opt2.merge(aisles, how='left', on='aisle_id')
opt=opt3.copy()

In [6]:
opt.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle
0,1,49302,1,1,Bulgarian Yogurt,120,16,dairy eggs,yogurt
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses
2,1,10246,3,0,Organic Celery Hearts,83,4,produce,fresh vegetables
3,1,49683,4,0,Cucumber Kirby,83,4,produce,fresh vegetables
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned goods,canned meat seafood


In [7]:
#find the number of aisles

print('# of unique aisles =',len(opt['aisle_id'].unique()))

# of unique aisles = 134


In [8]:
#one hot encode the aisles and create a dense matrix
ohe=OneHotEncoder()
aisle=ohe.fit_transform(opt['aisle_id'].values.reshape(-1,1))
aisle2=pd.DataFrame(aisle.toarray())
aisle3=aisle2.to_dense()
aisle_final=aisle3.copy()

In [9]:
#add order_id to the one hot encoded matrix
basket=opt[['order_id']].join(aisle_final)

In [10]:
basket.head()

,order_id,0,1,2,3,4,5,6,7,8,...,124,125,126,127,128,129,130,131,132,133
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
basket=basket.groupby('order_id').sum()

In [12]:
basket.head()

,0,1,2,3,4,5,6,7,8,9,...,124,125,126,127,128,129,130,131,132,133
order_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [13]:
basket[0].value_counts()

0.0    128598
1.0      2339
2.0       230
3.0        31
4.0        11
Name: 0, dtype: int64

In [14]:
#convert any values to be binary

def encode_units(x):
    if x<=0:
        return 0
    if x>=1:
        return 1

    

In [15]:
basket_sets=basket.applymap(encode_units)

In [16]:
basket_sets.columns

Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       124, 125, 126, 127, 128, 129, 130, 131, 132, 133],
      dtype='object', length=134)

In [17]:
basket_sets[0].value_counts()


0    128598
1      2611
Name: 0, dtype: int64

In [18]:
#extract frequent itemsets for association rule mining
frequent_itemsets=apriori(basket_sets,min_support=0.15, use_colnames=True)


In [19]:
frequent_itemsets.head()

,support,itemsets
0,0.237781,(20)
1,0.550099,(23)
2,0.450975,(82)
3,0.223750,(83)
4,0.170514,(90)


In [20]:
#creates the rules to filter.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(20),(23),0.237781,0.550099,0.160911,0.676720,1.230178,0.030108,1.391676
1,(23),(20),0.550099,0.237781,0.160911,0.292513,1.230178,0.030108,1.077361
2,(82),(23),0.450975,0.550099,0.327333,0.725833,1.319458,0.079252,1.640974
3,(23),(82),0.550099,0.450975,0.327333,0.595043,1.319458,0.079252,1.355760
4,(83),(23),0.223750,0.550099,0.151948,0.679099,1.234503,0.028864,1.401993
5,(23),(83),0.550099,0.223750,0.151948,0.276220,1.234503,0.028864,1.072494
6,(23),(119),0.550099,0.253405,0.185102,0.336488,1.327865,0.045704,1.125216
7,(119),(23),0.253405,0.550099,0.185102,0.730458,1.327865,0.045704,1.669130
8,(122),(23),0.383030,0.550099,0.287282,0.750025,1.363435,0.076577,1.799781
9,(23),(122),0.550099,0.383030,0.287282,0.522237,1.363435,0.076577,1.291371


In row 0, the row {20} is the antecedent and the row {23} is the consequent. Both antecedents and consequents can have multiple items. In other words, {23, 5} -> {20,112} is a valid rule.


Support is the relative frequency that the rules show up. In many instances, you may want to look for high support in order to make sure it is a useful relationship. However, there may be instances where a low support is useful if you are trying to find “hidden” relationships.
Confidence is a measure of the reliability of the rule. A confidence of .5 in the above example would mean that in 50% of the cases where a product from row 20 was purchased, the purchase also include a product from row 23. For product recommendation, a 50% confidence may be perfectly acceptable but in a medical situation, this level may not be high enough.

Lift is the ratio of the observed support to that expected if the two rules were independent (see wikipedia). The basic rule of thumb is that a lift value close to 1 means the rules were completely independent. Lift values > 1 are generally more “interesting” and could be indicative of a useful rule pattern.

This same process could be followed for products but would require an EC2 instance on AWS or similar.